In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder, MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd

# CSV 파일 불러오기
# 예: 'generated_data.csv'
real_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/VAE-CTAB-GAN/Real_Datasets/train_category_1.csv"
real = pd.read_csv(real_path, low_memory=False)

methods = {
    'smote-nc': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/base/smote-nc.csv',
    'tablegan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/tablegan.csv',
    'vae-tablegan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/tablegan/vae-tablegan.csv',
    'ctgan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/ctgan.csv',
    'vae-ctgan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctgan/vae-ctgan.csv',
    'ctabgan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/ctabgan.csv',
    'vae-ctabgan': 'C:/Users/GCU/Lending_club/Data_Analysis_lending-club/portfolios/ctabgan/vae-ctabgan.csv'
}

results = []

for method, path in tqdm(methods.items(), desc="Duplicated Ratio, Distinct Sample Ratio"):
    synth = pd.read_csv(path)

    # 전체 샘플 수
    total_samples = len(synth)

    # 고유 샘플 수
    unique_samples = synth.drop_duplicates().shape[0]

    # 중복 샘플 수
    duplicate_samples = total_samples - unique_samples

    # Duplicate Ratio (%)
    duplicate_ratio = (duplicate_samples / total_samples) * 100

    # Distinct Sample Ratio (%)
    distinct_sample_ratio = (unique_samples / total_samples) * 100

    results.append({
        "Methods": method,
        "Total Samples": total_samples,
        "Unique Samples": unique_samples,
        "Duplocate Samples": duplicate_samples,
        "Duplicate Ratio": duplicate_ratio,
        "Distinct Sample Ratio": distinct_sample_ratio
    })



Duplicated Ratio, Distinct Sample Ratio: 100%|██████████| 7/7 [00:08<00:00,  1.28s/it]


In [5]:
results

[{'Methods': 'smote-nc',
  'Total Samples': 540000,
  'Unique Samples': 540000,
  'Duplocate Samples': 0,
  'Duplicate Ratio': 0.0,
  'Distinct Sample Ratio': 100.0},
 {'Methods': 'tablegan',
  'Total Samples': 540000,
  'Unique Samples': 539708,
  'Duplocate Samples': 292,
  'Duplicate Ratio': 0.05407407407407407,
  'Distinct Sample Ratio': 99.94592592592593},
 {'Methods': 'vae-tablegan',
  'Total Samples': 540000,
  'Unique Samples': 539708,
  'Duplocate Samples': 292,
  'Duplicate Ratio': 0.05407407407407407,
  'Distinct Sample Ratio': 99.94592592592593},
 {'Methods': 'ctgan',
  'Total Samples': 540000,
  'Unique Samples': 540000,
  'Duplocate Samples': 0,
  'Duplicate Ratio': 0.0,
  'Distinct Sample Ratio': 100.0},
 {'Methods': 'vae-ctgan',
  'Total Samples': 540000,
  'Unique Samples': 539708,
  'Duplocate Samples': 292,
  'Duplicate Ratio': 0.05407407407407407,
  'Distinct Sample Ratio': 99.94592592592593},
 {'Methods': 'ctabgan',
  'Total Samples': 540000,
  'Unique Samples': 54